In [1]:
include("../../src/CoherentStructures.jl") #this is terrible but neccessary during development (?)

Main.CoherentStructures

In [2]:
using StreamMacros, Main.CoherentStructures
using Tensors, Arpack, JLD2, OrdinaryDiffEq
using Plots
using OrdinaryDiffEq
const ODE = OrdinaryDiffEq

OrdinaryDiffEq

In [3]:
JLD2.@load("Ocean_geostrophic_velocity.jld2");
UV = interpolateVF(Lon, Lat, Time, UT, VT);

In [4]:
# Parameters
grid_resolution        = 100
quadrature_order       = 1
t_initial              = minimum(Time)
t_end                  = t_initial+80
scale                  = 1;

In [5]:
LL, UR = (-4.0, -34.0), (6.0, -28.0)
ctx, _ = regularTriangularGrid((grid_resolution, Integer(floor((UR[2] - LL[2])/(UR[1] - LL[1])*grid_resolution))), 
        LL, UR, quadrature_order = quadrature_order);

In [6]:
function evaluate_on_grid(f,ctx)
    [f(node.x) for node in ctx.grid.nodes]
end

evaluate_on_grid (generic function with 1 method)

In [7]:
function l1_error(reference,solution)
    sum(sum([abs.(a .- b) for (a,b) in zip(reference,solution)])) / (2*length(reference))
end

l1_error (generic function with 1 method)

In [8]:
function l2_error(reference,solution)
    sum(sqrt.(sum([(a .- b).^2 for (a,b) in zip(reference,solution)]))) / (2*length(reference))
end

l2_error (generic function with 1 method)

In [9]:
function linfty_error(reference,solution)
    sum(maximum([(a .- b) for (a,b) in zip(reference,solution)]))
end

linfty_error (generic function with 1 method)

In [10]:
function T(solver, stepsize)
    return (x, param) -> flow(interp_rhs, x, [t_initial, t_end + scale*param], p=UV,
        dt=stepsize,adaptive=false,solver=solver)[end]
end

T (generic function with 1 method)

In [11]:
function Tdot_reference(solver, tolerance, param)
   return x -> UV(T(solver, tolerance)(x,param)...,t_end+param)*scale
end

Tdot_reference (generic function with 1 method)

In [12]:
function Tdot_autodiff(solver, stepsize, param)
    return x -> Main.CoherentStructures.parameter_autodiff(T(solver, stepsize), x, param)
end

Tdot_autodiff (generic function with 1 method)

In [13]:
function Tdot_finite_differences(solver, stepsize, param, diff)
    return x -> 
     (T(solver, stepsize)(x, param+diff) .- T(solver, stepsize)(x, param-diff)) ./ (2*diff)
end

Tdot_finite_differences (generic function with 1 method)

In [14]:
stepsize_range_BS5 = [5e-1,4.5e-1,4e-1,3.5e-1,3e-1,2.5e-1,2e-1,1.5e-1,1e-1,
                      9e-2,8.5e-2,8e-2,7.5e-2,7e-2,6.5e-2,6e-2,5.5e-2,
                      5e-2,4.5e-2,4e-2,3.5e-2,3e-2,2.5e-2,2e-2,1e-2,
                      9e-3,8e-3,7e-3,6e-3,5e-3,4e-3,3e-3,2e-3,1e-3];

In [ ]:
reference_BS5 = [evaluate_on_grid(Tdot_reference(OrdinaryDiffEq.BS5(),stepsize,0),ctx) 
    for stepsize in stepsize_range_BS5];

In [ ]:
autodiff_BS5    = [evaluate_on_grid(Tdot_autodiff(OrdinaryDiffEq.BS5(),stepsize,0),ctx) 
    for stepsize in stepsize_range_BS5]
FD_1eminus2_BS5 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.BS5(),stepsize,0,1e-2),ctx)
    for stepsize in stepsize_range_BS5]
FD_1eminus3_BS5 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.BS5(),stepsize,0,1e-3),ctx)
    for stepsize in stepsize_range_BS5]
FD_1eminus4_BS5 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.BS5(),stepsize,0,1e-4),ctx) 
    for stepsize in stepsize_range_BS5]
FD_1eminus5_BS5 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.BS5(),stepsize,0,1e-5),ctx)
    for stepsize in stepsize_range_BS5];
FD_1eminus6_BS5 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.BS5(),stepsize,0,1e-6),ctx)
    for stepsize in stepsize_range_BS5];
FD_1eminus7_BS5 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.BS5(),stepsize,0,1e-7),ctx)
    for stepsize in stepsize_range_BS5];

In [ ]:
l2_plot_BS5 = Plots.plot(stepsize_range_BS5,[l2_error(ref_stepsize,stepsize) 
        for (ref_stepsize,stepsize) in zip(reference_BS5,autodiff_BS5)], 
    xaxis=:log, yaxis=:log, marker=:dot, xflip=true, label="Autodiff BS5"
    ,fmt=:png,dpi=300,title="L2 error",legend=(0.1,0.25),legendfontsize=5)
Plots.plot!(stepsize_range_BS5,[l2_error(ref_stepsize,stepsize) 
        for (ref_stepsize,stepsize) in zip(reference_BS5,FD_1eminus2_BS5)], 
    marker=:dot, label="Finite Differences, δ=1e-2, BS5")
Plots.plot!(stepsize_range_BS5,[l2_error(ref_stepsize,stepsize) 
         for (ref_stepsize,stepsize) in zip(reference_BS5,FD_1eminus3_BS5)], 
    marker=:dot, label="Finite Differences, δ=1e-3, BS5")
Plots.plot!(stepsize_range_BS5,[l2_error(ref_stepsize,stepsize) 
         for (ref_stepsize,stepsize) in zip(reference_BS5,FD_1eminus4_BS5)], 
    marker=:dot, label="Finite Differences, δ=1e-4, BS5")
Plots.plot!(stepsize_range_BS5,[l2_error(ref_stepsize,stepsize) 
         for (ref_stepsize,stepsize) in zip(reference_BS5,FD_1eminus5_BS5)], 
    marker=:dot, label="Finite Differences, δ=1e-5, BS5")
Plots.plot!(stepsize_range_BS5,[l2_error(ref_stepsize,stepsize) 
         for (ref_stepsize,stepsize) in zip(reference_BS5,FD_1eminus6_BS5)], 
    marker=:dot, label="Finite Differences, δ=1e-6, BS5")
Plots.plot!(stepsize_range_BS5,[l2_error(ref_stepsize,stepsize) 
         for (ref_stepsize,stepsize) in zip(reference_BS5,FD_1eminus7_BS5)], 
    marker=:dot, label="Finite Differences, δ=1e-7, BS5")
Plots.plot!(stepsize_range_BS5,[x^(5.5) for x in stepsize_range_BS5], 
    linestyle=:dash,label="O(h^5.5)")

In [ ]:
stepsize_range_DP8 = [5e-1,4.5e-1,4e-1,3.5e-1,3e-1,2.5e-1,2e-1,1.5e-1,1e-1,
                      9e-2,8.5e-2,8e-2,7.5e-2,7e-2,6.5e-2,6e-2,5.5e-2,5e-2,4.5e-2,4e-2,3.5e-2,3e-2,2.5e-2,2e-2,1e-2,
                      9e-3,8e-3,7e-3,6e-3,5e-3,4e-3,3e-3,2e-3];

In [ ]:
reference_DP8 = [evaluate_on_grid(Tdot_reference(OrdinaryDiffEq.DP8(),stepsize,0),ctx) 
    for stepsize in stepsize_range_DP8];

In [ ]:
autodiff_DP8     = [evaluate_on_grid(Tdot_autodiff(OrdinaryDiffEq.DP8(),stepsize,0),ctx) 
    for stepsize in stepsize_range_DP8]
FD_1eminus2_DP8 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.DP8(),stepsize,0,1e-2),ctx)
    for stepsize in stepsize_range_DP8]
FD_1eminus3_DP8 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.DP8(),stepsize,0,1e-3),ctx)
    for stepsize in stepsize_range_DP8]
FD_1eminus4_DP8 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.DP8(),stepsize,0,1e-4),ctx) 
    for stepsize in stepsize_range_DP8]
FD_1eminus5_DP8 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.DP8(),stepsize,0,1e-5),ctx)
    for stepsize in stepsize_range_DP8];
FD_1eminus6_DP8 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.DP8(),stepsize,0,1e-6),ctx)
    for stepsize in stepsize_range_DP8];
FD_1eminus7_DP8 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.DP8(),stepsize,0,1e-7),ctx)
    for stepsize in stepsize_range_DP8];

In [ ]:
l2_plot_DP8 = Plots.plot(stepsize_range_DP8,[l2_error(ref_stepsize,stepsize) 
        for (ref_stepsize,stepsize) in zip(reference_DP8,autodiff_DP8)], 
    xaxis=:log, yaxis=:log, marker=:dot, xflip=true, label="Autodiff DP8"
    ,fmt=:png,dpi=300,title="L2 error",legend=(0.1,0.25),legendfontsize=5)
Plots.plot!(stepsize_range_DP8,[l2_error(ref_stepsize,stepsize) 
        for (ref_stepsize,stepsize) in zip(reference_DP8,FD_1eminus2_DP8)], 
    marker=:dot, label="Finite Differences, δ=1e-2, DP8")
Plots.plot!(stepsize_range_DP8,[l2_error(ref_stepsize,stepsize) 
         for (ref_stepsize,stepsize) in zip(reference_DP8,FD_1eminus3_DP8)], 
    marker=:dot, label="Finite Differences, δ=1e-3, DP8")
Plots.plot!(stepsize_range_DP8,[l2_error(ref_stepsize,stepsize) 
         for (ref_stepsize,stepsize) in zip(reference_DP8,FD_1eminus4_DP8)], 
    marker=:dot, label="Finite Differences, δ=1e-4, DP8")
Plots.plot!(stepsize_range_DP8,[l2_error(ref_stepsize,stepsize) 
         for (ref_stepsize,stepsize) in zip(reference_DP8,FD_1eminus5_DP8)], 
    marker=:dot, label="Finite Differences, δ=1e-5, DP8")
Plots.plot!(stepsize_range_DP8,[l2_error(ref_stepsize,stepsize) 
         for (ref_stepsize,stepsize) in zip(reference_DP8,FD_1eminus6_DP8)], 
    marker=:dot, label="Finite Differences, δ=1e-6, DP8")
Plots.plot!(stepsize_range_DP8,[l2_error(ref_stepsize,stepsize) 
         for (ref_stepsize,stepsize) in zip(reference_DP8,FD_1eminus7_DP8)], 
    marker=:dot, label="Finite Differences, δ=1e-7, DP8")
Plots.plot!(stepsize_range_DP8,[100*x^8 for x in stepsize_range_DP8], 
    linestyle=:dash,label="O(h^8)")

In [ ]:
stepsize_range_Vern7 = [5e-1,4.5e-1,4e-1,3.5e-1,3e-1,2.5e-1,2e-1,1.5e-1,1e-1,
                      9e-2,8.5e-2,8e-2,7.5e-2,7e-2,6.5e-2,6e-2,5.5e-2,5e-2,4.5e-2,4e-2,3.5e-2,3e-2,2.5e-2,2e-2,1e-2,
                      9e-3,8e-3,7e-3,6e-3,5e-3,4e-3,3e-3,2e-3,1e-3];

In [ ]:
reference_Vern7 = [evaluate_on_grid(Tdot_reference(OrdinaryDiffEq.Vern7(),stepsize,0),ctx) 
    for stepsize in stepsize_range_Vern7];

In [ ]:
autodiff_Vern7     = [evaluate_on_grid(Tdot_autodiff(OrdinaryDiffEq.Vern7(),stepsize,0),ctx) 
    for stepsize in stepsize_range_Vern7]
FD_1eminus2_Vern7 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.Vern7(),stepsize,0,1e-2),ctx)
    for stepsize in stepsize_range_Vern7]
FD_1eminus3_Vern7 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.Vern7(),stepsize,0,1e-3),ctx)
    for stepsize in stepsize_range_Vern7]
FD_1eminus4_Vern7 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.Vern7(),stepsize,0,1e-4),ctx) 
    for stepsize in stepsize_range_Vern7]
FD_1eminus5_Vern7 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.Vern7(),stepsize,0,1e-5),ctx)
    for stepsize in stepsize_range_Vern7];
FD_1eminus6_Vern7 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.Vern7(),stepsize,0,1e-6),ctx)
    for stepsize in stepsize_range_Vern7];
FD_1eminus7_Vern7 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.Vern7(),stepsize,0,1e-7),ctx)
    for stepsize in stepsize_range_Vern7];

In [ ]:
l2_plot_Vern7 = Plots.plot(stepsize_range_Vern7,[l2_error(ref_stepsize,stepsize) 
        for (ref_stepsize,stepsize) in zip(reference_Vern7,autodiff_Vern7)], 
    xaxis=:log, yaxis=:log, marker=:dot, xflip=true, label="Autodiff Vern7"
    ,fmt=:png,dpi=300,title="L2 error",legend=(0.1,0.25),legendfontsize=5)
Plots.plot!(stepsize_range_Vern7,[l2_error(ref_stepsize,stepsize) 
        for (ref_stepsize,stepsize) in zip(reference_Vern7,FD_1eminus2_Vern7)], 
    marker=:dot, label="Finite Differences, δ=1e-2, Vern7")
Plots.plot!(stepsize_range_Vern7,[l2_error(ref_stepsize,stepsize) 
         for (ref_stepsize,stepsize) in zip(reference_Vern7,FD_1eminus3_Vern7)], 
    marker=:dot, label="Finite Differences, δ=1e-3, Vern7")
Plots.plot!(stepsize_range_Vern7,[l2_error(ref_stepsize,stepsize) 
         for (ref_stepsize,stepsize) in zip(reference_Vern7,FD_1eminus4_Vern7)], 
    marker=:dot, label="Finite Differences, δ=1e-4, Vern7")
Plots.plot!(stepsize_range_Vern7,[l2_error(ref_stepsize,stepsize) 
         for (ref_stepsize,stepsize) in zip(reference_Vern7,FD_1eminus5_Vern7)], 
    marker=:dot, label="Finite Differences, δ=1e-5, Vern7")
Plots.plot!(stepsize_range_Vern7,[l2_error(ref_stepsize,stepsize) 
         for (ref_stepsize,stepsize) in zip(reference_Vern7,FD_1eminus6_Vern7)], 
    marker=:dot, label="Finite Differences, δ=1e-6, Vern7")
Plots.plot!(stepsize_range_Vern7,[l2_error(ref_stepsize,stepsize) 
         for (ref_stepsize,stepsize) in zip(reference_Vern7,FD_1eminus7_Vern7)], 
    marker=:dot, label="Finite Differences, δ=1e-7, Vern7")
Plots.plot!(stepsize_range_Vern7,[1000*x^8 for x in stepsize_range_Vern7], 
    linestyle=:dash,label="O(h^8)")